# the Solow model with fixed and exhaustible resources

Imports and set magics:

In [45]:
import numpy as np
from scipy import optimize
import sympy as sm

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# local modules
from modelproject import Solow
from modelproject import analytical

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Introduction

In this project we first start out by considering an extended Solow model with a fixed input.  
An example of a fixed input could be land

Later we extend the model to include an exhaistible resource as well.

## The standard Solow model

To remind the reader we here present relevant equations for the standard Solow model, which will later be used as a baseline to compare with the extended Solow model.

The production function is a Cobb-Douglas:
$$Y_t=K_t^\alpha (A_tL_t)^{1-\alpha},\quad \alpha>0$$
The transition equation for capital-output is given by:
$$z_{t+1}=\frac{(s_Y+(1-\delta)z_t)^{1-\alpha}}{((1+g)(1+n))^{1-\alpha}}z_t^\alpha$$
And the steady state for the capital-output ratio is:
$$z^\star=\frac{s_Y}{((1+g)(1+n))^{\frac{\beta}{1-\alpha}}-(1-\delta)}$$

# The Solow Model with land.

We first consider the Solow model with land.  
The model can be descripted by the following equations:

$$Y_t=K_t^\alpha  X^\kappa (A_tL_t)^{1-\alpha-\kappa}, \quad \alpha,\kappa>0$$
$$K_{t+1}=s_YY_t+(1-\delta)K_t$$
$$L_{t+1}=(1+n)L_t$$
$$A_{t+1}=(1+g)A_t$$

Here the variables are:
- $Y_t$: Production/Output
- $K_t$: Capital
- $X$: Land
- $L_t$: Labor
- $A_t$: Technology growth


$s_Y$ is the savings rate of capital, while n and g are the growth of labor and technology respectively.  

## Analytical solution

We evalute the steady state based on the capital-output ration:
$$z_t\equiv\frac{K_t}{Y_t}=\frac{k_t}{y_t}$$
We start from the next period and insert the prodution function:
$$z_{t+1}=\frac{K_{t+1}}{Y_{t+1}}=\frac{K_{t+1}}{K_{t+1}^\alpha X^\kappa(A_{t+1}L_{t+1})^{1-\alpha-\kappa}}=\frac{K_{t+1}^{1-\alpha}}{X^\kappa(A_{t+1}L_{t+1})^{1-\alpha-\kappa}}$$
We insert the capital accumulation equation:
$$z_{t+1}=\frac{(s_YY_t+(1-\delta)K_t)^{1-\alpha}}{X^\kappa(A_{t+1}L_{t+1})}=\frac{(s+z_t(1-\delta))^{1-\alpha}}{X^\kappa(A_{t+1}L_{t+1})^{1-\alpha-\kappa}}Y_t^{1-\alpha}$$
$$=\frac{(s+z_t(1-\delta))^{1-\alpha}}{((1+g)(1+n))^{1-\alpha-\kappa}X^\kappa(A_{t}L_{t})^{1-\alpha-\kappa}}Y_t^{1-\alpha}$$
Trick: We now multiply and divide by $K_t^\alpha$
$$=\frac{(s+z_t(1-\delta))^{1-\alpha}}{((1+g)(1+n))^{1-\alpha-\kappa}X^\kappa(A_{t}L_{t})^{1-\alpha-\kappa}}Y_t^{1-\alpha}\frac{K_t^\alpha}{K_t^\alpha}$$
$$=\frac{(s+z_t(1-\delta))^{1-\alpha}}{((1+g)(1+n))^{1-\alpha-\kappa}}\frac{Y_t}{Y_t}z_t^\alpha$$
$$=\frac{(s+z_t(1-\delta))^{1-\alpha}}{((1+g)(1+n))^{1-\alpha-\kappa}}z_t^\alpha$$

In steady state we have that $z_{t+1}=z_t=z$ and we can thus use sympy to solve for z

In [46]:
analytical_ext1 = analytical(ext1 = True, ext2 = False, do_print = True)

The solution to the model with extension 1 is: z = s_Y/(delta - 1 + ((g*n + g + n + 1)**(-alpha - kappa + 1))**(-1/(alpha - 1)))


Eq(z, s_Y/(delta - 1 + ((g*n + g + n + 1)**(-alpha - kappa + 1))**(-1/(alpha - 1))))

We can also solve for the standard model using sympy:

In [47]:
analytical_standard = analytical(ext1 = False, ext2 = False, do_print = True)

The solution to the model without any extensions is: z = s_Y/(delta - 1 + ((g*n + g + n + 1)**(1 - alpha))**(-1/(alpha - 1)))


Eq(z, s_Y/(delta - 1 + ((g*n + g + n + 1)**(1 - alpha))**(-1/(alpha - 1))))

## Numerical solution

We now want to numerically solve for when the steady state is reached, i.e. when the following holds:
$$z_{t+1}-z_t=\frac{(s+z_t(1-\delta))^{1-\alpha}}{((1+g)(1+n))^{1-\alpha-\kappa}}z_t^\alpha-z_t=0$$

In [48]:
model = Solow()     # Load in the Solow class
model.setup()    # Initialize the model

# Analytical solution for the model with land
z_sol_analytical_land = model.evaluate_ss(analytical_ext1, ext1 = True, ext2 = False, do_print = True)

# Numerical solution for the model with land
z_sol_bisect_land = model.solve_ss(method = 'bisect', ext1 = True, ext2 = False, do_print = True).root
z_sol_brentq_land = model.solve_ss(method = 'brentq', ext1 = True, ext2 = False, do_print = True).root



The analytical steady state is: z = 2.6856868721437137
The numerical steady state using bisect
      converged: True
           flag: converged
 function_calls: 48
     iterations: 46
           root: 2.6856868721433402
The numerical steady state using brentq
      converged: True
           flag: converged
 function_calls: 8
     iterations: 7
           root: 2.685686872143712


We see that both methods, bisect and brentq, both converge close to the analytical solution. Using numpy we can set a tolerance parameter to check if we accept the precision:

In [49]:
print(f'Is equal to analytical Steady State: {np.isclose(z_sol_bisect_land, z_sol_analytical_land, rtol=1e-5)}')
print(f'Is equal to analytical Steady State: {np.isclose(z_sol_brentq_land, z_sol_analytical_land, rtol=1e-5)}')

Is equal to analytical Steady State: True
Is equal to analytical Steady State: True


We can also check the speed of convergence for the two types of solvers:

In [50]:
# print('bisect')
# %timeit z_sol_bisect_land = model.solve_ss(method='bisect', ext1 = True, ext2 = False).root
# print('brentq')
# %timeit z_sol_brentq_land = model.solve_ss(method='brentq', ext1 = True, ext2 = False).root

So we see, that brentq is a lot quicker than bisect with a lower standard error as well.

# The Solow model with land and oil

We now extend the Solow model further by also including an exhaustible resource such as oil.
$$Y_t = K_t^\alpha(A_tL_t)^{1-\alpha-\kappa-\varepsilon}X^\kappa E_t^\varepsilon,\quad \alpha,\kappa,\varepsilon>0$$
$$K_{t+1}=s_YY_t+(1-\delta)K_t$$
$$L_{t+1}=(1+n)L_t$$
$$A_{t+1}=(1+g)A_t$$
$$E_t=s_E R_t$$
$$R_{t+1}=R_t-E_t$$

Here $R_t$ is the stock of oil, while $E_t$ is the usage of oil.  
In every period, the economy uses a share $s_E$ of the stock of oil.  
The stock of oil does not increase (e.g. no new deposits of oil are found)


## Analytical solution 

The derivation is very similar to above:
$$z_{t+1}=\frac{K_{t+1}}{Y_{t+1}}=\frac{s_Y Y_t+(1-\delta)K_t}{K_{t+1}^\alpha(A_{t+1}L_{t+1})^\beta X^\kappa E_{t+1}^\varepsilon}$$
We use that:
$$E_{t+1}=s_ER_{t+1}=s_E R_t(1-s_E)$$
Insert this, the producction function and capital accumulation:
$$z_{t+1}=\frac{\left(s_Y Y_t+K_t(1-\delta)\right)^{1-\alpha}}{\left((1+g)(1+n)\right)^\beta\left(A_tL_t\right)^\beta X^\kappa(s_E R_t(1-s_E))^\varepsilon}$$
$$=\frac{(\left(s_Y+z_t(1-\delta\right)^{1-\alpha})Y_t^{1-\alpha}}{\left((1+g)(1+n)\right)^\beta(1-s_E)^\varepsilon(A_tL_t)^\beta X^\kappa E_t^\varepsilon}\cdot\frac{K_t^\alpha}{K_t^\alpha}$$
$$=\frac{(s_Y+z_t(1-\delta)^{1-\alpha})}{((1+g)(1+n))^\beta(1-s_E)^\varepsilon}z_t^\alpha$$


Again, we can solve this for steady state using sympy:

In [51]:
analytical_ext2 = analytical(ext1 = False, ext2 = True, do_print = True)

The solution to the model with extension 2 is: z = s_Y/(delta - 1 + ((1 - s_E)**epsilon*(g*n + g + n + 1)**(-alpha - epsilon - kappa + 1))**(-1/(alpha - 1)))


Eq(z, s_Y/(delta - 1 + ((1 - s_E)**epsilon*(g*n + g + n + 1)**(-alpha - epsilon - kappa + 1))**(-1/(alpha - 1))))

## Numerical solution
The numerical solution here will closely follow the numerical analyzis above and thus few comments will be given.  
Instead we will compare all the different results in the next section 'Comparisons'

Again it needs to hold that:
$$z_{t+1}-z_t=\frac{(s_Y+z_t(1-\delta)^{1-\alpha})}{((1+g)(1+n))^\beta(1-s_E)^\varepsilon}z_t^\alpha-z_t=0$$

In [52]:
model = Solow()     # Load in the Solow class
model.setup()    # Initialize the model

# Analytical solution for the model with land and oil
z_sol_analytical_oil = model.evaluate_ss(analytical_ext2, ext1 = False, ext2 = True, do_print = True)

# Numerical solution for the model with land and oil
z_sol_bisect_oil = model.solve_ss(method = 'bisect', ext1 = False, ext2 = True, do_print = True).root
z_sol_brentq_oil = model.solve_ss(method = 'brentq', ext1 = False, ext2 = True, do_print = True).root


# Check tolerance
print(f'Is equal to analytical Steady State: {np.isclose(z_sol_bisect_land, z_sol_analytical_land, rtol=1e-5)}')
print(f'Is equal to analytical Steady State: {np.isclose(z_sol_brentq_land, z_sol_analytical_land, rtol=1e-5)}')

The analytical steady state is: z = 4.09490565332236
The numerical steady state using bisect
      converged: True
           flag: converged
 function_calls: 48
     iterations: 46
           root: 4.094905653321379
The numerical steady state using brentq
      converged: True
           flag: converged
 function_calls: 8
     iterations: 7
           root: 4.094905653322368
Is equal to analytical Steady State: True
Is equal to analytical Steady State: True


Again we see that both methods converge.  
We again check speed:

In [53]:
# print('bisect')
# %timeit z_sol_bisect_oil = model.solve_ss(method='bisect', ext1 = False, ext2 = True).root
# print('brentq')
# %timeit z_sol_brentq_oil = model.solve_ss(method='brentq', ext1 = False, ext2 = True).root

We again see, that it is brentq that performs best.

# Comparisons

Make detailed vizualizations of how your model changes with parameter values. 

Try to make an extension of the model. 

# Conclusion

Add concise conclusion. 